<a href="https://colab.research.google.com/github/dmlcks123/3_3_Other_Considerations_in_the_Regression_Model/blob/main/iris_logit_gridsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
print(iris.keys())
print(iris.DESCR)

# 3항 로짓모형(virginica, versicolor, setosa)

In [ ]:
import pandas as pd

iris_df = pd.DataFrame(iris.data, columns=iris.feature_names) # iris pandasdataframe에 저장 
species = pd.Series(iris.target, dtype = 'category') #범주형 변수로 지정
species = species.cat.rename_categories(iris.target_names)
iris_df['species'] = species

In [ ]:
iris_df.describe()

In [ ]:
#box-plot
iris_df.boxplot();

In [ ]:
iris_df.plot();
# 0~50, 50~100, 100이상 그룹으로 종을 구분할수 있겠다라는 input

In [ ]:
import seaborn as sns

sns.pairplot(iris_df, hue = 'species')

#setosa 구분은 잘되어있음

In [ ]:
#2 로지스틱 회귀분석을 통한 분류

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris.data[:, ], iris.target, # 2열3열의 특징행렬만 사용하는 예제임
                                                    test_size = 0.2, # test set 비율
                                                    random_state = 1, # 시드값
                                                    stratify = iris.target) # stratify는 계층적 데이터일때 넣어주는 옵션


In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver = 'lbfgs', multi_class = 'auto', C = 100.0,
                           random_state = 1) #multi class는 3개임, #C는 하이퍼파라미터
model.fit(X_train, y_train)

In [ ]:
print("학습 데이터 점수: {}",format(model.score(X_train, y_train)))
print("평가 데이터 점수: {}",format(model.score(X_test, y_test)))

In [ ]:
# 시각화
import numpy as np
X = np.vstack((X_train, X_test)) # vertical
y = np.hstack((y_train, y_test)) # horizontal

In [ ]:
from matplotlib.colors import ListedColormap
x1_min, x1_max = X[:, 0].min()-1, X[:, 0].max()+1
x2_min, x2_max = X[:, 1].min()-1, X[:, 1].max()+1
xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, 0.02),
                       np.arange(x2_min, x2_max, 0.02))
Z = model.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
Z = Z.reshape(xx1.shape)

species = ('Sectosa', 'Versicolour', 'Virginica')
markers = ('^', 'v', 's')
colors = ('blue', 'purple', 'red')
cmap = ListedColormap(colors[:len(np.unique(y))])
plt.contourf(xx1, xx2, Z, alpha = 0.3, cmap = cmap)
plt.xlim(xx1.min(), xx1.max())
plt.ylim(xx2.min(), xx2.max())

for idx, cl in enumerate(np.unique(y)):
  plt.scatter(x = X[y == cl, 0], y = X[y == cl, 1],
              alpha = 0.8, c = colors[idx],
              marker = markers[idx], label = species[cl],
              edgecolor = 'b')

# test-set
X_comb_test, y_comb_test = X[range(105, 150), :], y[range(105, 150)]
plt.scatter(X_comb_test[:, 0], X_comb_test[:, 1],
            c = 'yellow', edgecolor = 'k', alpha = 0.2,
            linewidth = 1, marker = 'o', s = 100, label = 'Test') 
plt.xlabel('Petal Length (cm)')
plt.ylabel('Petal Width (cm)')
plt.legend(loc = 'upper left')
plt.tight_layout();                                     

In [ ]:
import multiprocessing
from sklearn.model_selection import GridSearchCV

param_grid = [{'penalty': ['l1', 'l2'],
               'C': [2.0, 2.2, 2.4, 2.6, 2.8]}]

gs = GridSearchCV(estimator = LogisticRegression(), param_grid = param_grid,
                  scoring = 'accuracy', cv = 10, n_jobs = multiprocessing.cpu_count())
gs

In [ ]:
result = gs.fit(iris.data, iris.target)

In [ ]:
print(gs.best_estimator_)
print("최적 점수: {}".format(gs.best_score_))
print("최적 파라미터: {}".format(gs.best_params_))
pd.DataFrame(result.cv_results_)